In [1]:
import os 
import time
import datetime
import json

import math 
import warnings
warnings.filterwarnings("ignore")

import numpy as np 
import pandas as pd 
import tensorflow as tf 

In [2]:
from sklearn.model_selection import train_test_split

from tqdm.autonotebook import tqdm

In [3]:
from utils import *

# 定义参数配置

In [7]:
class Config(dict):
    def __init__(self, path=None):
        super().__init__()
        ## 定义训练参数
        self['num_epochs'] = 2 
        self['evaluateEvery'] = 100 
        self['checkpointEvery'] = 100 
        self['learningRate'] = 0.001 
        
        ## 定义模型参数
        self['embeddingSize'] = 200 
        self['dropoutProb'] = 0.5 
        self['l2RegLambda'] = 0.0 
        
        ## 定义基础参数
        self['sequenceLength'] = 1014 
        self['batch_size'] = 128 
        self['dataSource'] = path   # 表示训练集文件所在的目录，csv文件，包含2-3列，类别列为`emotion`或者`sentiment`
        self['stopWordSource'] = "../data/english"  # 表示停止词所在的文件
        self['numClasses'] = 1  
        self['train_size'] = 0.8   # 训练集和测试集比例
        self.threshold = 0.5 
        
        ## 保存模型参数
        self['checkpoint_dir'] = "../model/charCNN/imdb/checkpoint"
        self['summary_dir'] = "../model/charCNN/imdb/summary"
        self['max_to_keep'] = 5
        
        
        ## 专用于charCNN的一些参数
        ### 3个参数分别表示 卷积核的数量，卷积核的高度，池化的尺寸
        self['convLayers'] = [[256, 7, 4], 
                              [256, 7, 4], 
                              [256, 3, 4]]
        self['fcLayers'] = [512]
        self['epsilon'] = 1e-3  # BN层中防止分母为0而加入的极小值
        self['decay'] = 0.999   # BN层中用来计算滑动平均的值 
        self['alphabet'] = "abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^*()~`+-=<>{}[]"
        

## 定义数据生成类

In [8]:
class Dataset(object):
    def __init__(self, config):
        self._dataSource = config['dataSource']
        self._sequenceLength = config['sequenceLength']
        self._trainSize = config['train_size']
        
        self.trainReviews = []
        self.trainLabels = []
        self.evalReviews = []
        self.evalLabels = []
        
        self._alphabet = config['alphabet']
        self.charEmbedding = None
        self._charToIndex = {}
        self._indexToChar = {}
        
    def _readData(self, filePath):
        '''
        从csv文件中读取数据
        '''
        df = pd.read_csv(filePath)
        if "sentiment" in df.columns:
            labels = df["sentiment"].tolist()
        elif "emotion" in df.columns:
            labels = df["emotion"].tolist()
        
        self.labelList = list(set(labels))
        
        review = df["review"].tolist()
        reviews = [[char for char in line if char != " "] for line in review]
        return reviews, labels 
    
    def _reviewProcess(self, review, sequenceLength, charToIndex):
        '''
        将数据集中的每条评论用index表示
        wordToIndex中的"pad"对应index为0
        '''
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
        for i in range(sequenceLen):
            if review[i] in charToIndex:
                reviewVec[i] = charToIndex[review[i]]
            else:
                reviewVec[i] = charToIndex["UNK"]
        
        return reviewVec
    
    def _genVocabulary(self, reviews, prefix=""):
        '''
        生成字符向量和字符-索引映射字典
        '''
        chars = [char for char in self._alphabet]
        vocab, charEmbedding = self._getcharEmbedding(chars)
        self.charEmbedding = charEmbedding
        
        self._charToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToChar = dict(zip(list(range(len(vocab))), vocab))
        
        ## 将词汇-索引映射表保存为json数据
        basepath = "../data/charJson"
        if not os.path.exists(basepath):
            os.mkdir(basepath)
        with open(os.path.join(basepath, prefix+"_charToIndex.json"), "w", encoding="utf-8") as f: 
            json.dump(self._charToIndex, f)
        with open(os.path.join(basepath, prefix+"_indexToChar.json"), "w", encoding="utf-8") as f: 
            json.dump(self._indexToChar, f)
    
    def _getcharEmbedding(self, chars):
        '''
        将字符映射为one-hot向量
        '''
        alphabet = ["UNK"] + chars
        vocab = ["PAD"] + alphabet
        
        charEmbedding = []
        charEmbedding.append(np.zeros(len(alphabet), dtype="float32"))
        
        for i, alpha in enumerate(alphabet):
            onehot = np.zeros(len(alphabet), dtype="float32")
            # 生成每个字符对应的向量
            onehot[i] = 1 
            # 生成字符嵌入的向量矩阵
            charEmbedding.append(onehot)
        
        return vocab, np.array(charEmbedding)
    
    def _genTrainEvalData(self, x, y):
        '''
        生成训练集和验证集
        '''
        reviews = []
        labels = []
        
        # 遍历所有文本，将文本中的词转换为Indexbiaoshi 
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._charToIndex)
            reviews.append(reviewVec)
            labels.append([y[i]])
        
        #print(len(y))
            
        trainReviews, evalReviews, trainLabels, evalLabels = train_test_split(reviews, labels,
                                                                             train_size=self._trainSize,
                                                                             shuffle=True,
                                                                             random_state=2019,
                                                                             stratify=y)
        return trainReviews, trainLabels, evalReviews, evalLabels
    
    def dataGen(self, path, prefix=""):
        '''
        初始化训练集和验证集
        '''
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews, prefix)
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels)
        
        self.trainReviews = np.array(trainReviews)
        self.trainLabels = np.array(trainLabels)
        self.evalReviews = np.array(evalReviews)
        self.evalLabels = np.array(evalLabels)

In [9]:
import numpy as np

class DataGenerator:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.length = len(y)

    def next_batch(self, batch_size):
        '''
        生成每一个batch的数据集
        '''
        idx = np.random.choice(self.length, batch_size)
        #print(idx)
        #print(self.x[[1, 10, 9]])
        yield self.x[idx], self.y[idx]

    def iter_all(self, batch_size):
        '''
        按照batch迭代所有数据
        '''
        numBatches = self.length // batch_size
        for i in range(numBatches):
            start = i * batch_size
            end = start + batch_size
            batchX = np.array(self.x[start:end], dtype='int64')
            batchY = np.array(self.y[start:end], dtype="float32")
            yield batchX, batchY

# 定义charCNN模型和训练类

In [10]:
class CharCNN(BaseModel):
    def __init__(self, config, charEmbedding):
        super(CharCNN, self).__init__(config)
        self.charEmbedding = charEmbedding
        self.build_model()
        self.init_saver()
        
    def build_model(self):
        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, self.config['sequenceLength']], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        self.dropoutProb = tf.placeholder(tf.float32, name="dropoutProb")
        self.isTraining = tf.placeholder(tf.bool, name="isTraining")
        
        self.epsilon = self.config['epsilon']
        self.decay = self.config['decay']
        
        # 字符嵌入
        with tf.name_scope("embedding"):
            # 利用one-hot字符作为初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(self.charEmbedding, dtype=tf.float32, name="charEmbedding"),
                                name="W")
            # 获得字符嵌入
            self.embededChars = tf.nn.embedding_lookup(self.W, self.inputX)
            # 添加一个通道维度
            self.embededCharsExpand = tf.expand_dims(self.embededChars, -1)
            
        for i, cl in enumerate(self.config["convLayers"]):
            print("开始第", (i+1), "卷积层的处理")
            # 利用命名空间实现变量名复用
            with tf.name_scope(f"convLayer-{i+1}"):
                # 获取字符向量长度
                filterWidth = self.embededCharsExpand.get_shape()[2].value
                # filtershape=[height, width, in_channels, out_channels]
                filterShape = [cl[1], filterWidth, 1, cl[0]]
                
                stdv = 1 / math.sqrt(cl[0] * cl[1])
                # 初始化W和b的值
                wConv = tf.Variable(tf.random_uniform(filterShape, minval=-stdv, 
                                                     maxval=stdv),
                                   dtype=tf.float32, name="wConv")
                bConv = tf.Variable(tf.random_uniform(shape=[cl[0]], minval=-stdv,
                                                     maxval=stdv),
                                   name="bConv")
                
                conv = tf.nn.conv2d(self.embededCharsExpand, wConv, strides=[1,1,1,1],
                                   padding="VALID", name="Conv")
                # 加上偏差
                hConv = tf.nn.bias_add(conv, bConv)
                # 加上relu
                ## 得到的输出为[batch, seq_len-filter_size+1, 1, out_channels]
                hConv = tf.nn.relu(hConv)
                
                ## 卷积层中使用BN层
                #hConv = tf.layers.batch_normalization(hConv, training=self.isTraining)
                
                if cl[-1] is not None: 
                    # 维度是[batch, height, width, channel]
                    ksizeShape = [1, cl[2], 1, 1]
                    hPool = tf.nn.max_pool(hConv, ksize=ksizeShape, strides=ksizeShape,
                                          padding="VALID", name="pool")
                else:
                    hPool = hConv
                
                # 对维度进行转换，转换成卷积层的输入维度
                # 转换后的维度为 [batch, (seq_len-filter_size+1)/pool_size, channels, 1]
                self.embededCharsExpand = tf.transpose(hPool, [0, 1, 3, 2], name="transpose")
        
        with tf.name_scope("reshape"):
            ## 获取卷积层输出宽度
            fcDim = self.embededCharsExpand.get_shape()[1].value * self.embededCharsExpand.get_shape()[2].value
            self.inputReshape = tf.reshape(self.embededCharsExpand, [-1, fcDim])
        
        # 保存的是神经元的个数
        weights = [fcDim] + self.config['fcLayers']
        
        for i, fl in enumerate(self.config['fcLayers']):
            with tf.name_scope(f"fcLayer-{i+1}"):
                print("开始第", (i+1), "个全连接层的处理")
                stdv = 1 / math.sqrt(weights[i])
                
                # 定义全连接层初始化方法，均匀分布初始化w和b的值
                wFc = tf.Variable(tf.random_uniform([weights[i], fl], minval=-stdv, 
                                                   maxval=stdv),
                                 dtype=tf.float32, name="w")
                bFc = tf.Variable(tf.random_uniform(shape=[fl], minval=-stdv, 
                                                   maxval=stdv),
                                 dtype=tf.float32, name="b")
                self.fcInput = tf.nn.relu(tf.matmul(self.inputReshape, wFc) + bFc)
                ## 全连接之后使用batchNorm
                #self.fcInput = tf.layers.batch_normalization(self.fcInput, training=self.isTraining)
                
                with tf.name_scope("dropout"):
                    self.fcInputDrop = tf.nn.dropout(self.fcInput, rate=self.dropoutProb)
            self.inputReshape = self.fcInputDrop
        
        with tf.name_scope("outputLayer"):
            stdv = 1 / math.sqrt(weights[-1])
            self.logits = tf.layers.dense(self.inputReshape, self.config['numClasses'], name="dense",
                                              kernel_initializer=tf.random_uniform_initializer(
                                              minval=-stdv, maxval=stdv),
                                              bias_initializer=tf.random_uniform_initializer(
                                              minval=-stdv, maxval=stdv),
                                         activation=None)
            ## 进行二元分类
            self.predicitions = tf.cast(tf.greater_equal(self.predictions, 0.0),
                                      tf.float32, name="binaryPreds")
        
        with tf.name_scope("loss"):
            # 定义损失函数，对预测值进行softmax，再求交叉熵
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.logits, labels=self.inputY)
            self.loss = tf.reduce_mean(losses)
            
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            self.train_op = tf.train.AdamOptimizer(
                self.config["learningRate"]).minimize(self.loss, global_step=self.global_step_tensor)
    
    def init_saver(self):
        '''
        初始化用于保存模型的对象
        '''
        self.saver = tf.train.Saver(max_to_keep=self.config['max_to_keep'])
    

## 定义训练的类

In [11]:
class Trainer(BaseTrain):
    def __init__(self, sess, model, data, config, logger):
        '''
        这里的data要求是元组形式，data[0]表示train对象，data[1]表示eval对象
        '''
        super(Trainer, self).__init__(sess, model, data, config, logger)
        self.train = data[0]
        self.eval = data[1]
        
    def train_epoch(self):
        num_iter_per_epoch = self.train.length // self.config["batch_size"]
        
        for _ in tqdm(range(num_iter_per_epoch)):
            loss, metrics, step = self.train_step()
            #print(metrics)
            train_acc = metrics["accuracy"]
            train_f_score = metrics["f_beta"]
            ## 将训练过程中的损失写入
            summaries_dict = {"loss": loss,
                             "acc": np.array(train_acc), 
                             "f_score": np.array(train_f_score)}
            self.logger.summarize(step, summarizer="train", scope="train_summary",
                                 summaries_dict=summaries_dict)
            
            if step % self.config['evaluateEvery'] == 0: 
                print("Train —— Step: {} | Loss: {} | Acc: {} | F1_Score: {}".format(
                    step, loss, train_acc, train_f_score))
                ## 对测试集进行评估
                print("\nEvaluation: \n")
                eval_losses = []
                eval_true = []
                eval_pred = []
                for batchEval in self.eval.iter_all(self.config["batch_size"]):
                    loss, predictions = self.eval_step(batchEval[0], batchEval[1])
                    eval_losses.append(loss)
                    eval_true.extend(batchEval[-1])
                    eval_pred.extend(predictions)
                
                getMetric = Metric(np.array(eval_pred), np.array(eval_true), self.config)
                metrics = getMetric.get_metrics()
                
                prec_mean = np.round(metrics['precision'], 5)
                recall_mean = np.round(metrics['recall'], 5)
                loss_mean = np.round(np.mean(eval_losses), 5)
                time_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S %p")
                print("{} | Loss: {} | Precision: {} | Recall: {}".format(time_str, 
                                                                         loss_mean,
                                                                         prec_mean,
                                                                         recall_mean))
                summaries_dict = {"loss": np.array(loss_mean), 
                                 "precision": np.array(prec_mean),
                                 "recall": np.array(recall_mean)}
                self.logger.summarize(step, summarizer="test", scope="test_summary",
                                     summaries_dict=summaries_dict)
            if step % self.config["checkpointEvery"] == 0: 
                self.model.save(self.sess)
    
    def train_step(self):
        batch_x, batch_y = next(self.train.next_batch(self.config["batch_size"]))
        feed_dict = {self.model.inputX: batch_x, self.model.inputY: batch_y,
                    self.model.dropoutProb: self.config["dropoutProb"],
                    self.model.isTraining: True}
        
        _, loss, predictions, step = self.sess.run([self.model.train_op,
                                                   self.model.loss,
                                                   self.model.predictions, 
                                                   self.model.global_step_tensor],
                                                  feed_dict=feed_dict)
        #print(type(predictions))
        #print(type(batch_y))
        getMetric = Metric(predictions, batch_y, self.config)    
        metrics = getMetric.get_metrics()
            
        return loss, metrics, step 
    
    def eval_step(self, batch_x, batch_y):
        '''
        使用验证集进行测试
        '''
        #print(batch_x.shape, batch_y.shape)
        feed_dict = {self.model.inputX: batch_x, self.model.inputY: batch_y,
                    self.model.dropoutProb: 0.0, 
                    self.model.isTraining: False}
        loss, predictions = self.sess.run([self.model.loss, self.model.predictions],
                                         feed_dict=feed_dict)
            
        return loss, predictions

# 使用数据集进行训练和预测

## 使用IMDB数据集进行训练和预测

In [12]:
def main():
    # 实例化配置参数对象
    ## 执行训练数据文件名
    path = "../data/imdb/labeldTrain.csv"
    config = Config(path)
    
    create_dirs([config['summary_dir'], config["checkpoint_dir"]])
    
    data = Dataset(config)
    ## 生成训练集数据，第一个参数表示wordembedding文件所在的文件夹
    data.dataGen("../data/imdb/", prefix="imdb")
    train_X, train_y, eval_X, eval_y = data.trainReviews, data.trainLabels, data.evalReviews, data.evalLabels
    charEmbedding, labels = data.charEmbedding, data.labelList
    
    
    train_data = DataGenerator(train_X, train_y)
    eval_data = DataGenerator(eval_X, eval_y)
    pack_data = [train_data, eval_data]
    
    tf.reset_default_graph()
    ## 设置计算图的配置
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置GPU占用率
    
    sess = tf.Session(config=session_conf)
    
    ## 创建一个实例
    model = CharCNN(config, charEmbedding)
    logger = Logger(sess, config)
    
    trainer = Trainer(sess, model, pack_data, config, logger)
    
    trainer.train_all()

### 不加BN层

In [158]:
main()

开始第 1 卷积层的处理
开始第 2 卷积层的处理
开始第 3 卷积层的处理
开始第 1 个全连接层的处理

当前正处于第1次迭代


Train —— Step: 100 | Loss: 0.6839132308959961 | Acc: 0.54688 | F1_Score: 0

Evaluation: 

2019-08-05 19:46:35 PM | Loss: 0.6964799761772156 | Precision: 0.0 | Recall: 0.0
Saving model...
Model saved
Train —— Step: 200 | Loss: 0.6301621198654175 | Acc: 0.64062 | F1_Score: 0.34286

Evaluation: 

2019-08-05 19:46:43 PM | Loss: 0.657260000705719 | Precision: 0.74196 | Recall: 0.30624
Saving model...
Model saved
Train —— Step: 300 | Loss: 0.5283021330833435 | Acc: 0.73438 | F1_Score: 0.77334

Evaluation: 

2019-08-05 19:46:51 PM | Loss: 0.535290002822876 | Precision: 0.72054 | Recall: 0.77055
Saving model...
Model saved

当前正处于第2次迭代


Train —— Step: 400 | Loss: 0.3860490918159485 | Acc: 0.85938 | F1_Score: 0.85714

Evaluation: 

2019-08-05 19:46:59 PM | Loss: 0.4625299870967865 | Precision: 0.74846 | Recall: 0.87175
Saving model...
Model saved
Train —— Step: 500 | Loss: 0.3149944543838501 | Acc: 0.85938 | F1_Score: 0.86567

Evaluation: 

2019-08-05 19:47:07 PM | Loss: 0.4489099979400635 | Precision: 0.86432 | Recall: 0.69461
Saving model...
Model saved
Train —— Step: 600 | Loss: 0.23178991675376892 | Acc: 0.9375 | F1_Score: 0.94118

Evaluation: 

2019-08-05 19:47:15 PM | Loss: 0.4266499876976013 | Precision: 0.76839 | Recall: 0.90265
Saving model...
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Model saved

当前正处于第3次迭代


Train —— Step: 700 | Loss: 0.3035995662212372 | Acc: 0.85938 | F1_Score: 0.8

Evaluation: 

2019-08-05 19:47:23 PM | Loss: 0.3990499973297119 | Precision: 0.85214 | Recall: 0.80165
Saving model...
Model saved
Train —— Step: 800 | Loss: 0.29960718750953674 | Acc: 0.85938 | F1_Score: 0.85246

Evaluation: 

2019-08-05 19:47:31 PM | Loss: 0.3980399966239929 | Precision: 0.79981 | Recall: 0.87586
Saving model...
Model saved
Train —— Step: 900 | Loss: 0.3692077398300171 | Acc: 0.89062 | F1_Score: 0.86274

Evaluation: 

2019-08-05 19:47:41 PM | Loss: 0.4606800079345703 | Precision: 0.89203 | Recall: 0.67135
Saving model...
Model saved

当前正处于第4次迭代


Train —— Step: 1000 | Loss: 0.2907438278198242 | Acc: 0.875 | F1_Score: 0.85185

Evaluation: 

2019-08-05 19:47:50 PM | Loss: 0.425570011138916 | Precision: 0.87543 | Recall: 0.76872
Saving model...
Model saved
Train —— Step: 1100 | Loss: 0.14011931419372559 | Acc: 0.95312 | F1_Score: 0.96

Evaluation: 

2019-08-05 19:48:00 PM | Loss: 0.4216899871826172 | Precision: 0.83465 | Recall: 0.83702
Saving model...
Model saved
Train —— Step: 1200 | Loss: 0.197625994682312 | Acc: 0.90625 | F1_Score: 0.90625

Evaluation: 

2019-08-05 19:48:11 PM | Loss: 0.3999600112438202 | Precision: 0.85065 | Recall: 0.80045
Saving model...
Model saved

当前正处于第5次迭代


Train —— Step: 1300 | Loss: 0.1971798986196518 | Acc: 0.90625 | F1_Score: 0.89286

Evaluation: 

2019-08-05 19:48:22 PM | Loss: 0.5517299771308899 | Precision: 0.90089 | Recall: 0.6524
Saving model...
Model saved
Train —— Step: 1400 | Loss: 0.10200947523117065 | Acc: 0.96875 | F1_Score: 0.97143

Evaluation: 

2019-08-05 19:48:33 PM | Loss: 0.44863998889923096 | Precision: 0.82537 | Recall: 0.85767
Saving model...
Model saved
Train —— Step: 1500 | Loss: 0.06029757112264633 | Acc: 0.98438 | F1_Score: 0.98551

Evaluation: 

2019-08-05 19:48:44 PM | Loss: 0.49063000082969666 | Precision: 0.82006 | Recall: 0.85347
Saving model...
Model saved

当前正处于第6次迭代


Train —— Step: 1600 | Loss: 0.04910717159509659 | Acc: 0.98438 | F1_Score: 0.98508

Evaluation: 

2019-08-05 19:48:55 PM | Loss: 0.4995500147342682 | Precision: 0.81888 | Recall: 0.86366
Saving model...
Model saved
Train —— Step: 1700 | Loss: 0.09166111052036285 | Acc: 0.96875 | F1_Score: 0.96154

Evaluation: 

2019-08-05 19:49:06 PM | Loss: 0.5903900265693665 | Precision: 0.88622 | Recall: 0.73609
Saving model...
Model saved
Train —— Step: 1800 | Loss: 0.09356683492660522 | Acc: 0.96875 | F1_Score: 0.96774

Evaluation: 

2019-08-05 19:49:17 PM | Loss: 0.5260199904441833 | Precision: 0.81009 | Recall: 0.85063
Saving model...
Model saved


**不加BN层的情况下**

- <font size=3 color=red>**Precision: 0.81009, Recall: 0.85063**</font>

### 全连接层加上BN层

In [162]:
main()

开始第 1 卷积层的处理
开始第 2 卷积层的处理
开始第 3 卷积层的处理
开始第 1 个全连接层的处理

当前正处于第1次迭代


Train —— Step: 100 | Loss: 0.5864744186401367 | Acc: 0.65625 | F1_Score: 0.64516

Evaluation: 

2019-08-05 19:51:24 PM | Loss: 0.694159984588623 | Precision: 0.0 | Recall: 0.0
Saving model...
Model saved
Train —— Step: 200 | Loss: 0.40194952487945557 | Acc: 0.8125 | F1_Score: 0.83333

Evaluation: 

2019-08-05 19:51:32 PM | Loss: 0.8474599719047546 | Precision: 0.0 | Recall: 0.0
Saving model...
Model saved
Train —— Step: 300 | Loss: 0.34159210324287415 | Acc: 0.85938 | F1_Score: 0.86567

Evaluation: 

2019-08-05 19:51:41 PM | Loss: 1.4161499738693237 | Precision: 0.0 | Recall: 0.0
Saving model...
Model saved

当前正处于第2次迭代


Train —— Step: 400 | Loss: 0.30810922384262085 | Acc: 0.85938 | F1_Score: 0.85246

Evaluation: 

2019-08-05 19:51:52 PM | Loss: 0.7256900072097778 | Precision: 0.94435 | Recall: 0.16069
Saving model...
Model saved
Train —— Step: 500 | Loss: 0.45823484659194946 | Acc: 0.84375 | F1_Score: 0.84375

Evaluation: 

2019-08-05 19:52:05 PM | Loss: 0.8020899891853333 | Precision: 0.96408 | Recall: 0.2031
Saving model...
Model saved
Train —— Step: 600 | Loss: 0.2849304676055908 | Acc: 0.89062 | F1_Score: 0.89231

Evaluation: 

2019-08-05 19:52:18 PM | Loss: 3.457390069961548 | Precision: 0.0 | Recall: 0.0
Saving model...
Model saved

当前正处于第3次迭代


Train —— Step: 700 | Loss: 0.17484953999519348 | Acc: 0.95312 | F1_Score: 0.94915

Evaluation: 

2019-08-05 19:52:28 PM | Loss: 1.4201699495315552 | Precision: 0.56428 | Recall: 0.99267
Saving model...
Model saved
Train —— Step: 800 | Loss: 0.2737693190574646 | Acc: 0.89062 | F1_Score: 0.89855

Evaluation: 

2019-08-05 19:52:41 PM | Loss: 0.5051599740982056 | Precision: 0.754 | Recall: 0.88935
Saving model...
Model saved
Train —— Step: 900 | Loss: 0.2499176263809204 | Acc: 0.90625 | F1_Score: 0.90323

Evaluation: 

2019-08-05 19:52:54 PM | Loss: 1.2627700567245483 | Precision: 0.95258 | Recall: 0.24808
Saving model...
Model saved

当前正处于第4次迭代


Train —— Step: 1000 | Loss: 0.25301846861839294 | Acc: 0.92188 | F1_Score: 0.92308

Evaluation: 

2019-08-05 19:53:06 PM | Loss: 1.2031300067901611 | Precision: 0.94147 | Recall: 0.30944
Saving model...
Model saved
Train —— Step: 1100 | Loss: 0.1799057126045227 | Acc: 0.92188 | F1_Score: 0.91803

Evaluation: 

2019-08-05 19:53:18 PM | Loss: 0.8017299771308899 | Precision: 0.67712 | Recall: 0.93206
Saving model...
Model saved
Train —— Step: 1200 | Loss: 0.15340937674045563 | Acc: 0.9375 | F1_Score: 0.94872

Evaluation: 

2019-08-05 19:53:28 PM | Loss: 0.5666700005531311 | Precision: 0.75832 | Recall: 0.878
Saving model...
Model saved

当前正处于第5次迭代


Train —— Step: 1300 | Loss: 0.25575926899909973 | Acc: 0.875 | F1_Score: 0.84615

Evaluation: 

2019-08-05 19:53:41 PM | Loss: 0.8224700093269348 | Precision: 0.68257 | Recall: 0.94131
Saving model...
Model saved
Train —— Step: 1400 | Loss: 0.09212815761566162 | Acc: 0.95312 | F1_Score: 0.95384

Evaluation: 

2019-08-05 19:53:54 PM | Loss: 0.6971399784088135 | Precision: 0.72818 | Recall: 0.90369
Saving model...
Model saved
Train —— Step: 1500 | Loss: 0.16601687669754028 | Acc: 0.90625 | F1_Score: 0.91429

Evaluation: 

2019-08-05 19:54:06 PM | Loss: 0.8026800155639648 | Precision: 0.69389 | Recall: 0.92798
Saving model...
Model saved

当前正处于第6次迭代


Train —— Step: 1600 | Loss: 0.041238460689783096 | Acc: 1.0 | F1_Score: 1.0

Evaluation: 

2019-08-05 19:54:19 PM | Loss: 2.581429958343506 | Precision: 0.55109 | Recall: 0.99472
Saving model...
Model saved
Train —— Step: 1700 | Loss: 0.0845390260219574 | Acc: 0.9375 | F1_Score: 0.93549

Evaluation: 

2019-08-05 19:54:32 PM | Loss: 0.6754299998283386 | Precision: 0.74887 | Recall: 0.8765
Saving model...
Model saved
Train —— Step: 1800 | Loss: 0.19554844498634338 | Acc: 0.9375 | F1_Score: 0.94444

Evaluation: 

2019-08-05 19:54:42 PM | Loss: 0.7096899747848511 | Precision: 0.77888 | Recall: 0.77662
Saving model...
Model saved


**训练不稳定**

### 卷积层加上BN

In [167]:
main()

开始第 1 卷积层的处理
开始第 2 卷积层的处理
开始第 3 卷积层的处理
开始第 1 个全连接层的处理

当前正处于第1次迭代


Train —— Step: 100 | Loss: 0.775078535079956 | Acc: 0.4375 | F1_Score: 0.35715

Evaluation: 

2019-08-05 19:57:09 PM | Loss: 1.06318998336792 | Precision: 0.0 | Recall: 0.0
Saving model...
Model saved
Train —— Step: 200 | Loss: 0.48879241943359375 | Acc: 0.76562 | F1_Score: 0.7541

Evaluation: 

2019-08-05 19:57:19 PM | Loss: 1.0161399841308594 | Precision: 0.49969 | Recall: 0.99743
Saving model...
Model saved
Train —— Step: 300 | Loss: 0.48301830887794495 | Acc: 0.71875 | F1_Score: 0.70968

Evaluation: 

2019-08-05 19:57:32 PM | Loss: 6.239630222320557 | Precision: 0.4998 | Recall: 1.0
Saving model...
Model saved

当前正处于第2次迭代


Train —— Step: 400 | Loss: 0.48465025424957275 | Acc: 0.76562 | F1_Score: 0.77612

Evaluation: 

2019-08-05 19:57:46 PM | Loss: 7.591770172119141 | Precision: 0.4998 | Recall: 1.0
Saving model...
Model saved
Train —— Step: 500 | Loss: 0.46064239740371704 | Acc: 0.82812 | F1_Score: 0.81967

Evaluation: 

2019-08-05 19:57:59 PM | Loss: 7.97737979888916 | Precision: 0.4998 | Recall: 1.0
Saving model...
Model saved
Train —— Step: 600 | Loss: 0.4038413166999817 | Acc: 0.82812 | F1_Score: 0.84507

Evaluation: 

2019-08-05 19:58:10 PM | Loss: 4.503910064697266 | Precision: 0.4998 | Recall: 1.0
Saving model...
Model saved

当前正处于第3次迭代


Train —— Step: 700 | Loss: 0.3730517625808716 | Acc: 0.875 | F1_Score: 0.86667

Evaluation: 

2019-08-05 19:58:23 PM | Loss: 5.685400009155273 | Precision: 0.4998 | Recall: 1.0
Saving model...
Model saved
Train —— Step: 800 | Loss: 0.33008190989494324 | Acc: 0.82812 | F1_Score: 0.85333

Evaluation: 

2019-08-05 19:58:36 PM | Loss: 3.277400016784668 | Precision: 0.50445 | Recall: 1.0
Saving model...
Model saved
Train —— Step: 900 | Loss: 0.30713731050491333 | Acc: 0.84375 | F1_Score: 0.83333

Evaluation: 

2019-08-05 19:58:48 PM | Loss: 2.877959966659546 | Precision: 0.51273 | Recall: 0.99967
Saving model...
Model saved

当前正处于第4次迭代


Train —— Step: 1000 | Loss: 0.28179311752319336 | Acc: 0.89062 | F1_Score: 0.90909

Evaluation: 

2019-08-05 19:59:01 PM | Loss: 1.2377300262451172 | Precision: 0.5695 | Recall: 0.99251
Saving model...
Model saved
Train —— Step: 1100 | Loss: 0.24694417417049408 | Acc: 0.89062 | F1_Score: 0.88888

Evaluation: 

2019-08-05 19:59:15 PM | Loss: 0.43316999077796936 | Precision: 0.78111 | Recall: 0.87856
Saving model...
Model saved
Train —— Step: 1200 | Loss: 0.17582647502422333 | Acc: 0.92188 | F1_Score: 0.92754

Evaluation: 

2019-08-05 19:59:30 PM | Loss: 2.5766000747680664 | Precision: 0.39744 | Recall: 0.01666
Saving model...
Model saved

当前正处于第5次迭代


Train —— Step: 1300 | Loss: 0.2051563709974289 | Acc: 0.89062 | F1_Score: 0.88888

Evaluation: 

2019-08-05 19:59:49 PM | Loss: 1.0683300495147705 | Precision: 0.60023 | Recall: 0.98883
Saving model...
Model saved
Train —— Step: 1400 | Loss: 0.1436210721731186 | Acc: 0.95312 | F1_Score: 0.95652

Evaluation: 

2019-08-05 20:00:07 PM | Loss: 2.443010091781616 | Precision: 0.53142 | Recall: 0.99853
Saving model...
Model saved
Train —— Step: 1500 | Loss: 0.11777602136135101 | Acc: 0.96875 | F1_Score: 0.9697

Evaluation: 

2019-08-05 20:00:24 PM | Loss: 1.9643800258636475 | Precision: 0.54178 | Recall: 0.99842
Saving model...
Model saved

当前正处于第6次迭代


Train —— Step: 1600 | Loss: 0.1417352706193924 | Acc: 0.9375 | F1_Score: 0.9375

Evaluation: 

2019-08-05 20:00:40 PM | Loss: 3.0454699993133545 | Precision: 0.80726 | Recall: 0.05482
Saving model...
Model saved
Train —— Step: 1700 | Loss: 0.3656095862388611 | Acc: 0.84375 | F1_Score: 0.85714

Evaluation: 

2019-08-05 20:00:52 PM | Loss: 0.9760199785232544 | Precision: 0.67047 | Recall: 0.947
Saving model...
Model saved
Train —— Step: 1800 | Loss: 0.08302639424800873 | Acc: 0.95312 | F1_Score: 0.95384

Evaluation: 

2019-08-05 20:01:06 PM | Loss: 0.5607699751853943 | Precision: 0.84827 | Recall: 0.76045
Saving model...
Model saved


**训练不稳定，模型变化比较频繁**

## 使用Yelps数据集训练

In [13]:
def main():
    # 实例化配置参数对象
    ## 执行训练数据文件名
    path = "../data/yelps/yelps_test.csv"
    config = Config(path)
    config["summary_dir"] = "../model/charCNN/yelps/summary"
    config["checkpoint_dir"] = "../model/charCNN/yelps/checkpoint"
    
    
    create_dirs([config['summary_dir'], config["checkpoint_dir"]])
    
    data = Dataset(config)
    ## 生成训练集数据，第一个参数表示wordembedding文件所在的文件夹
    data.dataGen("../data/yelps/", prefix="yelps")
    train_X, train_y, eval_X, eval_y = data.trainReviews, data.trainLabels, data.evalReviews, data.evalLabels
    charEmbedding, labels = data.charEmbedding, data.labelList
    
    
    train_data = DataGenerator(train_X, train_y)
    eval_data = DataGenerator(eval_X, eval_y)
    pack_data = [train_data, eval_data]
    
    tf.reset_default_graph()
    ## 设置计算图的配置
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth = True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置GPU占用率
    
    sess = tf.Session(config=session_conf)
    
    ## 创建一个实例
    model = CharCNN(config, charEmbedding)
    logger = Logger(sess, config)
    
    trainer = Trainer(sess, model, pack_data, config, logger)
    
    trainer.train_all()

### 不加BN层

In [14]:
main()

Instructions for updating:
Colocations handled automatically by placer.
开始第 1 卷积层的处理
开始第 2 卷积层的处理
开始第 3 卷积层的处理
开始第 1 个全连接层的处理
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.

当前正处于第1次迭代


Train —— Step: 100 | Loss: 0.6253750324249268 | Acc: 0.63281 | F1_Score: 0.75132

Evaluation: 

2019-08-05 20:10:45 PM | Loss: 0.5958899855613708 | Precision: 0.67787 | Recall: 0.94746
Saving model...
Model saved
Train —— Step: 200 | Loss: 0.44988584518432617 | Acc: 0.8125 | F1_Score: 0.86363

Evaluation: 

2019-08-05 20:11:37 PM | Loss: 0.4607900083065033 | Precision: 0.82442 | Recall: 0.84355
Saving model...
Model saved
Train —— Step: 300 | Loss: 0.4822368919849396 | Acc: 0.78125 | F1_Score: 0.82927

Evaluation: 

2019-08-05 20:13:03 PM | Loss: 0.41710999608039856 | Precision: 0.89644 | Recall: 0.79979
Saving model...
Model saved
Train —— Step: 400 | Loss: 0.36840933561325073 | Acc: 0.83594 | F1_Score: 0.88135

Evaluation: 

2019-08-05 20:14:34 PM | Loss: 0.4070099890232086 | Precision: 0.79589 | Recall: 0.96258
Saving model...
Model saved
Train —— Step: 500 | Loss: 0.32802414894104004 | Acc: 0.85938 | F1_Score: 0.8875

Evaluation: 

2019-08-05 20:16:11 PM | Loss: 0.34981998801231384

Train —— Step: 3800 | Loss: 0.34356141090393066 | Acc: 0.85156 | F1_Score: 0.87249

Evaluation: 

2019-08-05 21:13:57 PM | Loss: 0.26603999733924866 | Precision: 0.92321 | Recall: 0.90798
Saving model...
Model saved
Train —— Step: 3900 | Loss: 0.3008839786052704 | Acc: 0.85156 | F1_Score: 0.88051

Evaluation: 

2019-08-05 21:15:33 PM | Loss: 0.26370999217033386 | Precision: 0.92888 | Recall: 0.90154
Saving model...
Model saved
Train —— Step: 4000 | Loss: 0.1505025029182434 | Acc: 0.96094 | F1_Score: 0.97382

Evaluation: 

2019-08-05 21:17:18 PM | Loss: 0.2612600028514862 | Precision: 0.89094 | Recall: 0.9521
Saving model...
Model saved
Train —— Step: 4100 | Loss: 0.2880496382713318 | Acc: 0.85156 | F1_Score: 0.88623

Evaluation: 

2019-08-05 21:19:00 PM | Loss: 0.2755599915981293 | Precision: 0.93699 | Recall: 0.88413
Saving model...
Model saved
Train —— Step: 4200 | Loss: 0.20999237895011902 | Acc: 0.89062 | F1_Score: 0.91139

Evaluation: 

2019-08-05 21:20:42 PM | Loss: 0.26407000422

Train —— Step: 4500 | Loss: 0.24300113320350647 | Acc: 0.90625 | F1_Score: 0.93182

Evaluation: 

2019-08-05 21:25:32 PM | Loss: 0.2543399930000305 | Precision: 0.90586 | Recall: 0.93744
Saving model...
Model saved
Train —— Step: 4600 | Loss: 0.19490298628807068 | Acc: 0.91406 | F1_Score: 0.93785

Evaluation: 

2019-08-05 21:27:11 PM | Loss: 0.2558000087738037 | Precision: 0.90768 | Recall: 0.93383
Saving model...
Model saved
Train —— Step: 4700 | Loss: 0.2730187177658081 | Acc: 0.88281 | F1_Score: 0.9162

Evaluation: 

2019-08-05 21:28:44 PM | Loss: 0.25845998525619507 | Precision: 0.91784 | Recall: 0.92011
Saving model...
Model saved
Train —— Step: 4800 | Loss: 0.24061965942382812 | Acc: 0.90625 | F1_Score: 0.93333

Evaluation: 

2019-08-05 21:30:22 PM | Loss: 0.25690001249313354 | Precision: 0.8954 | Recall: 0.94871
Saving model...
Model saved
Train —— Step: 4900 | Loss: 0.21891529858112335 | Acc: 0.88281 | F1_Score: 0.90322

Evaluation: 

2019-08-05 21:31:59 PM | Loss: 0.2812100052

2019-08-05 22:31:52 PM | Loss: 0.24969999492168427 | Precision: 0.91255 | Recall: 0.9313
Saving model...
Model saved
Train —— Step: 8400 | Loss: 0.17576271295547485 | Acc: 0.89062 | F1_Score: 0.92473

Evaluation: 

2019-08-05 22:33:30 PM | Loss: 0.2735700011253357 | Precision: 0.93942 | Recall: 0.88082
Saving model...
Model saved
Train —— Step: 8500 | Loss: 0.1797901839017868 | Acc: 0.9375 | F1_Score: 0.95555

Evaluation: 

2019-08-05 22:35:16 PM | Loss: 0.25227999687194824 | Precision: 0.91995 | Recall: 0.92259
Saving model...
Model saved
Train —— Step: 8600 | Loss: 0.20872198045253754 | Acc: 0.90625 | F1_Score: 0.93103

Evaluation: 

2019-08-05 22:36:58 PM | Loss: 0.2534100115299225 | Precision: 0.8979 | Recall: 0.95067
Saving model...
Model saved
Train —— Step: 8700 | Loss: 0.3278994560241699 | Acc: 0.875 | F1_Score: 0.91011

Evaluation: 

2019-08-05 22:38:36 PM | Loss: 0.2506299912929535 | Precision: 0.90794 | Recall: 0.93781
Saving model...
Model saved
Train —— Step: 8800 | Loss: 